In [ ]:
from datetime import datetime
import pyspark
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
def drop_table_if_exists(tbl_path) -> None:
    
    from delta import DeltaTable

    try:
        # Load the Delta table
        delta_table = DeltaTable.forPath(spark, tbl_path)

        # Delete the Delta table
        delta_table.delete()
    except Exception as e:
        pass

In [ ]:
def create_or_replace_delta_table(df, tbl_path) -> None:
    try:
        df.write.format("delta").mode("overwrite").save(tbl_path)
    except Exception as e:
        df.write.format("delta").save(tbl_path)

In [ ]:
def build_src_table() -> None:
    schema = StructType([
        StructField("name", StringType(), True),
        StructField("age", IntegerType(), True),
        StructField("hire_date", DateType(), True)
    ])

    data = [
        ("Matt", 20, datetime(2022,8,19)),
        ("Bill", 35, datetime(2023,4,15)),
        ("Nancy", 57, datetime(2022,4,23)),
        ("Rachel", 19, datetime(2021,6,7)),
    ]

    df = spark.createDataFrame(data, schema=schema)
    create_or_replace_delta_table(df, "/home/jovyan/work/src_ppl")

In [ ]:
build_src_table()